# Scrapping de données météo

Dans ce notebook, on cherche à récupérer les données météo de plusieurs communes, afin de rendre pertinente une régression de la consommation d'énergie en fonction de l'indice de performance du bâtiment et de la météo.

Cette analyse s'inscrit dans une dimension bien plus locale que l'analyse précédente, qui portait sur une échelle départementale et non communale.

In [1]:
!pip install -q lxml

In [16]:
#!pip install -q lxml

import bs4
import lxml
import pandas as pd
import urllib
from tqdm.auto import tqdm

from urllib import request

In [3]:
""" Création du dictionnaire Racine """

data_all = {
    i : {j : {} for j in range(1,13)} for i in range(2011,2019)
}

In [4]:
""" Scrapping du site infoclimat.fr et organisation des données sous forme de dictionnaire """
# Il faut optimiser la création en ne créant qu'une seule fois le Df lié aux villes

for i in tqdm(range(len(data_all))):
    for y in data_all :
        for m in data_all[y] :
            url_test = 'https://www.infoclimat.fr/stations-meteo/analyses-mensuelles.php?mois=' + str(m) + '&annee=' + str(y)
            raw_text = request.urlopen(url_test).read()
            page = bs4.BeautifulSoup(raw_text) #,'lxml')
            tableau = page.find('table', {'id' : 'tableau-releves'})
            rows = tableau.find_all('tr')
    
            df = {
        'Villes' : [rows[i].find('a').text for i in range(1,len(rows))], #[rows[i].find_all('div')[0].text for i in range(1,len(rows))], #[rows[i].find('a').text for i in range(1,len(rows))],
        'Température minimale extrême du mois' : [rows[i].find('div').text for i in range(1,len(rows))],
        'Moyenne des températures minimales du mois' : [rows[i].find_all('td')[2].text for i in range(1,len(rows))],
        'Température moyenne du mois' : [rows[i].find_all('td')[3].text for i in range(1,len(rows))],
        'Moyenne des températures maximales du mois' : [rows[i].find_all('td')[4].text for i in range(1,len(rows))],
        'Température maximale extrême du mois' : [rows[i].find_all('td')[5].find('div').text for i in range(1,len(rows))],
        'Cumul de précipitation du mois' : [rows[i].find_all('td')[6].text for i in range(1,len(rows))],
        "Heure d'ensolleiment du mois" : [rows[i].find_all('td')[7].text for i in range(1,len(rows))],
        'Rafale maximale du mois' : [rows[i].find_all('td')[8].text for i in range(1,len(rows))] 
                }
    
            data_all[y][m] = df

  0%|          | 0/8 [00:00<?, ?it/s]

## Création d'une liste de ville

On garde pour la suite le nom des stations utiles. Ces noms nous permettront de filtrer les communes dont nous devons étudier les DPE.

In [39]:
df['Villes']

['Abbeville (80)',
 'Agde (34)',
 'Agde - Le Grau (34)',
 'Agen - La Garenne (47)',
 "Aigrefeuille-d'Aunis (17)",
 "Aigrefeuille-d'Aunis - Château-d'eau (17)",
 'AILLEVILLERS (70)',
 'Aix-en-Provence (13)',
 "Aix-les-Bains (Capitainerie d'Aix Grand Lac) (73)",
 'Aizenay (85)',
 "Ajaccio - Campo dell'Oro (2A)",
 'Albi-Le Séquestre (81)',
 'Alençon - Valframbert (61)',
 'Alenya - Mas Blanc (66)',
 'Almenêches (61)',
 'Ambérieu (01)',
 'Ambérieu-en-Bugey (01)',
 'Ambès (33)',
 'Amiens - Dury (80)',
 'Amuré (79)',
 'Ancey (21)',
 'Angers - Marcé (49)',
 'Angers-Beaucouzé (49)',
 'Anglars-Saint-Félix (12)',
 'Aniane (34)',
 'Annecy-Meythet (74)',
 'ANNECY-MEYTHET (FR)',
 'Annonay (07)',
 'Annot (04)',
 'Aouste-sur-Sye (Saint-Pierre) (26)',
 'Arc-sous-Cicon (25)',
 'Arces (17)',
 'Arcizans-Avant (65)',
 'Argenteuil (95)',
 'Armentières (59)',
 'Arnas (69)',
 'Arthun (42)',
 'Artonne (63)',
 'Arvert (1) (17)',
 'Arvert (2) (17)',
 'Asnières-sur-Oise (95)',
 'Atuona (987)',
 'AUBENAS SA (7)',


In [28]:
#Création d'un Data Frame à partir de la colonne "Villes" du dictionnaire précédent
stations=pd.DataFrame(df['Villes'],columns=['Ville'])
stations

,Ville
0,Abbeville (80)
1,Agde (34)
2,Agde - Le Grau (34)
3,Agen - La Garenne (47)
4,Aigrefeuille-d'Aunis (17)
...,...
873,[MAE] Lycée Roz Glas - QUIMPERLE (29)
874,[MAE] Lycée Saint Exupéry - LA ROCHELLE (17)
875,[MAE] Lycée Victor Louis - TALENCE (33)
876,[MAE] Lycée Victor Louis - TALENCE (33)


In [29]:
#On ne garde que les stations dans les villes et pas celle de stations de "Météo à l'école" (MAE)
liste_communes=stations[~stations['Ville'].str[0].isin(['['])]
liste_communes.head(20)

,Ville
0,Abbeville (80)
1,Agde (34)
2,Agde - Le Grau (34)
3,Agen - La Garenne (47)
4,Aigrefeuille-d'Aunis (17)
5,Aigrefeuille-d'Aunis - Château-d'eau (17)
6,AILLEVILLERS (70)
7,Aix-en-Provence (13)
8,Aix-les-Bains (Capitainerie d'Aix Grand Lac) (73)
9,Aizenay (85)


On voit qu'en plus par ville il peut il y avoir plusieurs stations (correspondant à plusieurs quartiers). On va donc essayer de supprimer les parties qui ne dénotent pas de la ville dans les noms de stations.

In [40]:
# Méthode très naïve
List=[]
for ville in liste_communes['Ville']:
    i=0
    mot=''
    while i < len(ville) and ville[i] != '(' :
        mot+=ville[i]
        i+=1
    List.append(mot)
List

['Abbeville ',
 'Agde ',
 'Agde - Le Grau ',
 'Agen - La Garenne ',
 "Aigrefeuille-d'Aunis ",
 "Aigrefeuille-d'Aunis - Château-d'eau ",
 'AILLEVILLERS ',
 'Aix-en-Provence ',
 'Aix-les-Bains ',
 'Aizenay ',
 "Ajaccio - Campo dell'Oro ",
 'Albi-Le Séquestre ',
 'Alençon - Valframbert ',
 'Alenya - Mas Blanc ',
 'Almenêches ',
 'Ambérieu ',
 'Ambérieu-en-Bugey ',
 'Ambès ',
 'Amiens - Dury ',
 'Amuré ',
 'Ancey ',
 'Angers - Marcé ',
 'Angers-Beaucouzé ',
 'Anglars-Saint-Félix ',
 'Aniane ',
 'Annecy-Meythet ',
 'ANNECY-MEYTHET ',
 'Annonay ',
 'Annot ',
 'Aouste-sur-Sye ',
 'Arc-sous-Cicon ',
 'Arces ',
 'Arcizans-Avant ',
 'Argenteuil ',
 'Armentières ',
 'Arnas ',
 'Arthun ',
 'Artonne ',
 'Arvert ',
 'Arvert ',
 'Asnières-sur-Oise ',
 'Atuona ',
 'AUBENAS SA ',
 'Aubenas-Vals ',
 'Auch-Lamothe ',
 'Audinghen - Cap Gris-Nez ',
 'Augy-sur-Aubois ',
 'Aulnois-sous-Laon ',
 'Auribeau-sur-Siagne ',
 'Aurillac ',
 'Autrans ',
 'Autun ',
 'Auvers-le-Hamon ',
 'Auxerre-Perrigny ',
 'Auzevill

In [5]:
""" Conversion en DataFrame pour exporter """

df_all = pandas.DataFrame(data_all)

In [6]:
""" Exportation au format json """

df_all.to_json('data_base.json')

In [9]:
""" Normalisation des variables suivant la nomenclature de la source """

df_all.rename({'Villes' : 'ville', 
           'Température minimale extrême du mois' : 'tnn',
           'Moyenne des températures minimales du mois' : 'tnm', 
           'Température moyenne du mois' : 'tmm', 
           'Moyenne des températures maximales du mois' : 'txm', 
           'Température maximale extrême du mois' : 'txx', 
           'Cumul de précipitation du mois' : 'rr', 
           "Heure d'ensolleiment du mois" : 'ens', 
           'Rafale maximale du mois' : 'rafale'},axis=1,inplace = True)

In [10]:
class Meteo :
    def __init__(self):
        self.data_all = {
                i : {j : {} for j in range(1,13)} for i in range(2011,2019)
            }

    def scrap(self):
        for y in self.data_all :
            for m in self.data_all[y] :
                url_test = 'https://www.infoclimat.fr/stations-meteo/analyses-mensuelles.php?mois=' + str(m) + '&annee=' + str(y)
                raw_text = request.urlopen(url_test).read()
                page = bs4.BeautifulSoup(raw_text,'lxml')
                tableau = page.find('table', {'id' : 'tableau-releves'})
                rows = tableau.find_all('tr')
        
                df = {
            'Villes' : [rows[i].find('a').text for i in range(1,len(rows))], #[rows[i].find_all('div')[0].text for i in range(1,len(rows))], #[rows[i].find('a').text for i in range(1,len(rows))],
            'Température minimale extrême du mois' : [rows[i].find('div').text for i in range(1,len(rows))],
            'Moyenne des températures minimales du mois' : [rows[i].find_all('td')[2].text for i in range(1,len(rows))],
            'Température moyenne du mois' : [rows[i].find_all('td')[3].text for i in range(1,len(rows))],
            'Moyenne des températures maximales du mois' : [rows[i].find_all('td')[4].text for i in range(1,len(rows))],
            'Température maximale extrême du mois' : [rows[i].find_all('td')[5].find('div').text for i in range(1,len(rows))],
            'Cumul de précipitation du mois' : [rows[i].find_all('td')[6].text for i in range(1,len(rows))],
            "Heure d'ensolleiment du mois" : [rows[i].find_all('td')[7].text for i in range(1,len(rows))],
            'Rafale maximale du mois' : [rows[i].find_all('td')[8].text for i in range(1,len(rows))] 
                    }
        
                self.data_all[y][m] = df

    def export(self):
        self.df_all = pandas.DataFrame(self.data_all)
        self.df_all.to_json('data_base.json')

    def rename(self):
        self.df_all = pandas.DataFrame(self.data_all)
        self.df_all.rename({'Villes' : 'ville', 
           'Température minimale extrême du mois' : 'tnn',
           'Moyenne des températures minimales du mois' : 'tnm', 
           'Température moyenne du mois' : 'tmm', 
           'Moyenne des températures maximales du mois' : 'txm', 
           'Température maximale extrême du mois' : 'txx', 
           'Cumul de précipitation du mois' : 'rr', 
           "Heure d'ensolleiment du mois" : 'ens', 
           'Rafale maximale du mois' : 'rafale'},axis=1,inplace = True)
    

### Récupération des données de Diagnostics de Performance Energétique (DPE)

On commence par programmer une fonction qui permet de récupérer des données à partir d'une APL.

In [ ]:
def get_dpe_from_url(api_url):
    '''Argument:
        api_url (string) : url de l'API voulue
    Sortie:
        pandas.DataFrame : dataframe
    '''
    req=requests.get(api_url)
    wb=req.json()
    df = pd.json_normalize(wb["results"])
    dpe = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs = 4326)
    dpe = dpe.dropna(subset = ['longitude', 'latitude'])
    return dpe

Il est trop coûteux de récupérer l'intégralité des données de DPE, et nous ne disposons de données météo que pour certaines communes. L'idée est donc de ne récupérer les données de DPE que pour les communes voulues et de les agréger dans une unique table de données.

In [ ]:
def get_dpe_dataset(api_root, communes):
    '''
    Args : 
    Returns : 
    '''
    